In [4]:
import os
import sys
path_to_this_notebook = os.path.abspath('.')
path_to_project = path_to_this_notebook[:path_to_this_notebook.find('note')]
sys.path.append(path_to_project)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [5]:
path_to_data = path_to_project + '/data/pecanstreet/'

In [6]:

df = pd.read_csv(path_to_data + '1minute_data_newyork.csv')
date = df['localminute'].apply(lambda x: x[:10])
time = df['localminute'].apply(lambda x: x[11:16])
df['date'] = date
df['time'] = time
df = df.fillna(0)
solar = df['solar'] + df['solar2']
car = df['car1'] + df['car2']
columns = ['dataid', 'date', 'time', 'grid',]
df = df[columns]
df['solar'] = solar
df['car'] = car

In [7]:
for dataid in df['dataid'].unique():
    mask = df['dataid'].isin([dataid])
    df_masked = df[mask]
    df_masked['time_float'] = df_masked['time'].apply(lambda x: 60 * int(x[:2]) + int(x[3:]))
    df_masked = df_masked.sort_values(by=['date', 'time_float'])
    df_masked.reset_index(drop=True, inplace=True)
    df_masked.drop(columns=['time_float'], inplace=True)
    df_masked['usage'] = df_masked.solar + df_masked.grid
    df_masked.to_csv(path_to_data + 'houses/%s.csv' % dataid, index=None)

/var/folders/gx/t12fwbln6l9dsbhpcps1_8gh0000gn/T/ipykernel_21599/685970345.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_masked['time_float'] = df_masked['time'].apply(lambda x: 60 * int(x[:2]) + int(x[3:]))


In [8]:
metadata_df = pd.read_csv(path_to_data + '/metadata_raw.csv', )[1:].reset_index(drop=True)
metadata_df = metadata_df.fillna(False)
metadata_df = metadata_df.astype({'dataid': str, 'pv': bool})

metadata_df.head()

,dataid,active_record,building_type,city,state,egauge_1min_min_time,egauge_1min_max_time,egauge_1min_data_availability,egauge_1s_min_time,egauge_1s_max_time,...,program_baseline,program_energy_internet_demo,program_lg_appliance,program_verizon,program_ccet_group,program_civita_group,program_shines,energy_storage_system,energy_storage_system_l1,energy_storage_system_l2
0,2836,False,Single-Family Home 001 (Master),Austin,Texas,False,False,False,False,False,...,False,yes,False,False,False,False,False,False,False,False
1,2743,False,Single-Family Home 001 (Master),Austin,Texas,False,False,False,False,False,...,False,yes,False,False,False,False,False,False,False,False
2,5323,False,Single-Family Home 001 (Master),Austin,Texas,False,False,False,False,False,...,yes,yes,False,False,False,False,False,False,False,False
3,8560,False,Single-Family Home 001 (Master),Austin,Texas,False,False,False,False,False,...,yes,yes,False,False,False,False,False,False,False,False
4,3313,False,Single-Family Home 001 (Master),Austin,Texas,False,False,False,False,False,...,yes,yes,False,False,False,False,False,False,False,False


In [9]:
houses = [f[:-4] for f in os.listdir(path_to_data + '/houses/') ]
metadata_dict = {hid: {} for hid in houses}
for hid in houses:
    df = pd.read_csv(path_to_data + '/houses/%s.csv' % hid)
    max_pv = df['solar'].max()
    given_val = float(metadata_df['total_amount_of_pv'][metadata_df['dataid'] == hid].iloc[0])
    has_pv = True if max_pv > 0 else False
    dates = df['date'].unique()
    
    metadata_dict[hid]['solar_rated_power'] = max_pv
    metadata_dict[hid]['has_pv'] = has_pv
    metadata_dict[hid]['dates'] = dates

In [10]:
with open(path_to_data + 'metadata_dict.pickle', 'wb') as f:
    pickle.dump(metadata_dict, f, protocol=pickle.HIGHEST_PROTOCOL)